In [ ]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import shapely.geometry
import numpy as np
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
def calculate_ndvi(file_path, nir_band=4, red_band=3):
    with rasterio.open(file_path) as src:
        nir = src.read(nir_band).astype(float)
        red = src.read(red_band).astype(float)
        print(src.shape)
        np.seterr(divide='ignore', invalid='ignore')
        ndvi = (nir - red) / (nir + red)
        return ndvi

def calculate_ndwi(file_path, nir_band=4, green_band=2):
    with rasterio.open(file_path) as src:
        nir = src.read(nir_band).astype(float)
        green = src.read(green_band).astype(float)
        np.seterr(divide='ignore', invalid='ignore')
        ndwi = (green - nir) / (green + nir)
        return ndwi
    
    
def save_index_to_tiff(index, src, output_filename):
    with rasterio.open(
        output_filename,
        'w',
        driver='GTiff',
        height=index.shape[0],
        width=index.shape[1],
        count=1,
        dtype=index.dtype,
        crs=src.crs,
        transform=src.transform,
    ) as dst:
        dst.write(index, 1)



In [ ]:
def process_and_analyze_images(base_folder, bbox_geojson, output_folder_name):
    bbox_geom = gpd.GeoDataFrame.from_features([{'type': 'Feature', 'properties': {}, 'geometry': bbox_geojson}], crs='EPSG:4326')

    output_folder = os.path.join(base_folder, output_folder_name)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    ndvi_data = []
    ndwi_data = []

    for subdir, dirs, files in os.walk(base_folder):
        for file in files:
            if file.endswith('.tif') and file and not file.startswith('._'):
                file_path = os.path.join(subdir, file)
                print("Processing file:", file_path)

                try:
                    with rasterio.open(file_path) as src:
                        date_str = file.split('_')[0]
                        date_str = file.split('_')[0]
                        year = date_str[:4]
                        month = date_str[4:6]
                        day = date_str[6:8]
                        
                        year_folder = os.path.join(output_folder, year)
                        if not os.path.exists(year_folder):
                            os.makedirs(year_folder)

                        name, ext = os.path.splitext(file)

                        # Calculate and save NDVI & NDWI for the entire TIFF
                        ndvi = calculate_ndvi(file_path)
                        ndwi = calculate_ndwi(file_path)

                        ndvi_file = os.path.join(year_folder, f'{name}_NDVI{ext}')
                        ndwi_file = os.path.join(year_folder, f'{name}_NDWI{ext}')

                        save_index_to_tiff(ndvi, src, ndvi_file)
                        save_index_to_tiff(ndwi, src, ndwi_file)

                        print(f"Saved NDVI & NDWI TIFF for {file}")

                        # Crop the image
                        src_crs = src.crs if src.crs else 'EPSG:4326'
                        bbox_geom = bbox_geom.to_crs(crs=src_crs)

                        intersection = bbox_geom.unary_union.intersection(shapely.geometry.box(*src.bounds))
                        if not intersection.is_empty:
                            out_image, out_transform = mask(src, shapes=[intersection], crop=True)
                            out_meta = src.meta.copy()
                            out_meta.update({
                                "driver": "GTiff",
                                "height": out_image.shape[1],
                                "width": out_image.shape[2],
                                "transform": out_transform
                            })

                            output_file = os.path.join(year_folder, f'{name}_clipped{ext}')
                            with rasterio.open(output_file, "w", **out_meta) as dest:
                                dest.write(out_image)

                            avg_ndvi = np.nanmean(ndvi)
                            avg_ndwi = np.nanmean(ndwi)

                            ndvi_data.append({'year': year, 'month': month, 'day': day, 'avg_ndvi': avg_ndvi})
                            ndwi_data.append({'year': year, 'month': month, 'day': day, 'avg_ndwi': avg_ndwi})

                            print(f"Processed and saved clipped image for {file}")
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")

    ndvi_df = pd.DataFrame(ndvi_data)
    ndwi_df = pd.DataFrame(ndwi_data)

    ndvi_df.to_csv(os.path.join(output_folder, 'average_ndvi.csv'), index=False)
    ndwi_df.to_csv(os.path.join(output_folder, 'average_ndwi.csv'), index=False)



In [ ]:
base_folder = '/Volumes/CedarLakes/Cedar Lakes Data/Aerial Imagery/clip files'
bbox_geojson = {
    "type": "Polygon",
    "coordinates": [[[-95.521295,28.830356],[-95.532078,28.825407],[-95.525845,28.814612],[-95.514663,28.820121],[-95.521295,28.830356]]]
}
output_folder_name = 'processed_images'

process_and_analyze_images(base_folder, bbox_geojson, output_folder_name)


In [ ]:


# File paths
ndvi_csv_path = '/Volumes/CedarLakes/Cedar Lakes Data/Aerial Imagery/clip files/processed_images/average_ndvi.csv'
ndwi_csv_path = '/Volumes/CedarLakes/Cedar Lakes Data/Aerial Imagery/clip files/processed_images/average_ndwi.csv'

# Load the data
ndvi_df = pd.read_csv(ndvi_csv_path)
ndwi_df = pd.read_csv(ndwi_csv_path)

# Convert to datetime and set as index
ndvi_df['date'] = pd.to_datetime(ndvi_df[['year', 'month', 'day']])
ndwi_df['date'] = pd.to_datetime(ndwi_df[['year', 'month', 'day']])
ndvi_df.set_index('date', inplace=True)
ndwi_df.set_index('date', inplace=True)

# Sort the dataframes
ndvi_df.sort_index(inplace=True)
ndwi_df.sort_index(inplace=True)

# Plotting
plt.figure(figsize=(12, 6))

# Plot NDVI
plt.plot(ndvi_df.index, ndvi_df['avg_ndvi'], label='Average NDVI', color='green')

# Plot NDWI
plt.plot(ndwi_df.index, ndwi_df['avg_ndwi'], label='Average NDWI', color='blue')

plt.xlabel('Date')
plt.ylabel('Index Value')
plt.title('Average NDVI and NDWI Over Time')
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()
